In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 리스트
word_list = []
# with를 쓰면 파일을 읽고 자동으로 close 해줌
with open('./word.txt','r',encoding='UTF-8') as file_data:
    # 파일에서 가져온걸 한라인씩 읽어와라
    for line in file_data:
        # 한 라인씩 읽어온 걸 리스트에 붙임
        word_list.append(line.strip())

In [20]:
def get_word_dic(lis, s_print=False):

    # 리스트
    result_list = []


    for li in lis:

        # 딕셔너리
        result_dic = {}
        
        li_a = li.find('a', 't_blue1')
        # 순번
        if li_a.find('sup') != None:  # sup이 있으면
            r_seq = li_a.find('sup').text.strip()
            result_dic['r_seq'] = r_seq
            li_a.find('sup').decompose()
        
        # 글자
        r_word = li_a.text.strip() 
        ## 딕셔너리
        result_dic['r_word'] = r_word

        # 링크
        r_link = 'https://stdict.korean.go.kr' + li.find('a', 't_blue1')['href']
        ## 딕셔너리
        result_dic['r_link'] = r_link

        # 한자
        r_chi = li.find('span','t_gray hanja_font').text.strip()
        ## 딕셔너리
        result_dic['r_chi'] = r_chi

        # 검색어의 뜻
        r_des = li.find('font','dataLine').text.strip()
        ## 딕셔너리
        result_dic['r_des'] = r_des

        # 품사
        li_dt = li.find('dt')
        li_dt.find('a','t_blue1').decompose()
        li_dt.find('span').decompose()
        li_dt.find('span').decompose()

        r_pos=li_dt.text.strip()
        ## 딕셔너리
        result_dic['r_pos'] = r_pos

        # 리스트에 넣기
        result_list.append(result_dic)

    # 결과
    # end 로 줄바꿈을 없앰
    # 2번째 변수 값이 있으면(True) 실행되고, 없거나 False이면 실행되지않음
    if s_print:
        for result in result_list:
            print(result['r_word'], end='')
            # result 안에 r_seq 키 값이 있다면 
            if 'r_seq' in result.keys(): 
                print(result['r_seq'], end='')
            print(result['r_chi'], end='')
            print(result['r_pos'], end='')
            print(result['r_des'])
            print('  ',result['r_link'])
            print('-----------------')
        
    return result_list

In [21]:
r_list = []

for word in word_list:

    url = "https://stdict.korean.go.kr/search/searchResult.do?pageSize=10&searchKeyword={}".format(word)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    ul = soup.find('ul', 'result')
    # print(ul)

    # 검색결과가 없을때
    if ul != None:
        lis = ul.find_all('li')
        # 가져온 값을 누적해서 r_list에 저장
        r_list += get_word_dic(lis, False)
    else:
        print('검색결과가 없습니다!')

### 몽고DB 접속

In [22]:
from pymongo import MongoClient

In [23]:
client = MongoClient('localhost',27017)

In [24]:
# db 접속. 없으면 생성
db = client['kor_dic_db']

In [25]:
# 컬렉션에 접속. 없으면 생성
kor_dic_coll = db['kor_dic_coll']

In [26]:
# 값 넣기
kor_dic_coll.insert_many(r_list)

In [27]:
for data_row in kor_dic_coll.find({}, {'_id':0}):
    print(data_row)
    print('-------------------------')

{'r_seq': '1', 'r_word': '학원', 'r_link': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3', 'r_chi': '(學員)', 'r_des': '공부하는 인원.', 'r_pos': '「명사」'}
-------------------------
{'r_seq': '2', 'r_word': '학원', 'r_link': 'https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3', 'r_chi': '(學院)', 'r_des': '『교육』 일정한 목적ㆍ교과 과정ㆍ설비ㆍ제도 및 법규에 의하여 계속적으로 학생에게 교육을 실시하는 기관.=학교.', 'r_pos': '「명사」'}
-------------------------
{'r_seq': '3', 'r_word': '학원', 'r_link': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3', 'r_chi': '(學園)', 'r_des': '『교육』 학교 및 기타 교육 기관을 통틀어 이르는 말.', 'r_pos': '「명사」'}
-------------------------
{'r_seq': '4', 'r_word': '학원', 'r_link': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3', 'r_chi': '(學園)', 'r_des': '『문학』 1919년에 발간된 최초의 학생 잡지. 창가, 소설 따위가 발표되어 소년ㆍ소녀들에게 처음으로 문학적인 분위기에 접하는 계기를 만들어 주었다.', 'r_pos': '「명사」'}
-------------------------
{'r_word': '학교', 'r_li